In [ ]:
import pandas as pd

# Add the parent directory of the 'utils' directory to the Python path
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


import utils.utils as u
import utils.utils_people as up

-------------------------------------------------------------------------------------------------------------------

# Format verifications and manual assignations

-------------------------------------------------------------------------------------------------------------------

#### Load dataset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset_DF = pd.read_json('../data_catastici/data_post-processing/pipeline_steps/catastici_1741_step8.json')
people_data = dataset_DF[dataset_DF['owner_code'] == "PPL"]
people_data.head()

### CHECK - Family name missing for people-owned entities

In [ ]:
mask = (people_data['owner_family_name'].str.len() == 0)
people_data.loc[mask, 'owner_code'] = "1C_VRF"

In [ ]:
people_data[people_data['owner_code'] == "1C_VRF"]

### CHECK - Every entry has same number of FIRST and LAST NAMES

This consideres '-' (missing value) as a valid first or last name format.

In [ ]:
filtered_data = people_data[(people_data['owner_family_name'].str.len() > 0) & (people_data['owner_first_name'].str.len() > 0)]
mask = (filtered_data['owner_family_name'].str.split('|').apply(len) != filtered_data['owner_first_name'].str.split('|').apply(len))
people_data.loc[people_data.index.isin(filtered_data[mask].index), 'owner_code'] = "2C_VRF"

In [ ]:
people_data[people_data['owner_code'] == "2C_VRF"]

### CHECK - Very long family names

In [ ]:
mask = people_data['owner_family_name'].str.split('|').apply(lambda x: any(len(name.split(' ')) > 4 for name in x))
people_data.loc[mask, 'owner_code'] = "4C_VRF"

In [ ]:
people_data[people_data['owner_code'] == "4C_VRF"]

## Assign all entries to verify

In [ ]:
people_data.loc[people_data['owner_code'].str.contains('C_VRF'), 'owner_code'] = "C_VRF"

In [ ]:
dataset_DF.update(people_data)

In [ ]:
dataset_DF.to_json('../data_catastici/data_post-processing/pipeline_steps/catastici_1741_STD.json', orient='records', index=['uidx'])

In [ ]:
with pd.ExcelWriter('../data_catastici/data_post-processing/catastici_1741_STD.xlsx') as writer:
    dataset_DF.to_excel(writer, sheet_name='catastici_1741', index_label='uidx') 

-------------------------------------------------------------------------------

In [ ]:
import time
print("Current time:", time.strftime("%H:%M:%S", time.localtime()))